Tasa atractiva mínima (MARR)
===

Notas de clase sobre ingeniería economica avanzada usando Python

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia  

**Software utilizado**
>Este es un documento interactivo escrito como un notebook de [Jupyter](http://jupyter.org/) , en el cual se presenta un tutorial sobre finanzas corporativas usando Python. Los notebooks de Jupyter permiten incoporar simultáneamente código, texto, gráficos y ecuaciones. El código presentado en este notebook puede ejecutarse en los sistemas operativos Linux y OS X.  

>Haga click [aquí](https://github.com/jdvelasq/guias-de-instalacion) para obtener instrucciones detalladas sobre como instalar Jupyter en Windows y Mac OS X.

>Descargue la última versión de este documento a su disco duro; luego, carguelo y ejecutelo en línea en Try [Jupyter!](https://try.jupyter.org/)



#### Contenido

>

**Bibliografía**

> * [1] SAS/ETS 14.1 User's Guide, 2015.   
* [2] **hp** 12c platinum financial calculator. User's guide. 
* [3] HP Business Consultant II Owner's manual.
* [4] C.S. Park and G.P. Sharp-Bette. Advanced Engineering Economics. John Wiley & Sons, Inc., 1990.

# Problema del costo de capital

* A medida que se invierte más capital los rendimientos obtenidos son menores (es más dificil acceder a inversiones con rentabilidades altas).

* A medida que se presta más capital los interses son más altos (es más dificil acceder a créditos baratos)

* Si se tiene un proyecto cuyos fondos provienen del aporte de los socios y de diferentes esquemas de financiación, ¿cómo se calculá el costo de dichos fondos?.

<img src="images/wacc-explain.png" width=850>

# Caso práctico

Una compañía tiene las siguientes fuentes de financiamiento:

* Un total de \$ 4000 por la emisión de 4.000 acciones. Se espera un dividendo de \$ 0.25 por acción para los próximos años.

* Un préstamo bancario (Préstamo 1) de \$ 2.000. El préstamo se paga en 4 cuotas iguales a capital más intereses sobre el saldo total de deuda liquidados a una tasa efectiva de interés del 20%. En el momento del desembolso se cobró una comisión bancaria de \$ 50.

* Un préstamo bancario (Préstamo 2) de \$ 1.000 con descuento de 24 puntos. El préstamo se paga en 4 cuotas totales iguales que incluyen intereses más capital. La tasa de interés es del 20%.

* La venta de un bono con pago principal de \$ 5.000, el cual fue vendido por \$ 4.000. El capital se dedimirá en 4 periodos y se pagarán intereses a una tasa del 7%. El bono tiene un costo de venta de \$ 50.

El impuesto de renta es del 30%.

**Solución**

In [1]:
import cashflow as cf

In [2]:
##
## Se tienen cuatro fuentes de capital con diferentes costos
## sus datos se almacenarar en las siguientes listas:
##
monto   = [0] * 4
interes = [0] * 4

In [3]:
## emision de acciones
## --------------------------------------
monto[0]   = 4000
interes[0] = 0.25 / 1.0  # tasa de descueto de la accion

In [4]:
## préstamo 1.
## -------------------------------------------------------
## 
credito1 = cf.fixedLoan(amount     = 2000,     # monto
                        rate       =    0.20,  # tasa de interés
                        life       =    4,     # número de cuotas
                        orgpoints  = 50/2000)  # costos de originación
credito1           


   t          Beginning         Total      Interest     Principal        Ending
              Principal       Payment       Payment       Payment     Principal
--------------------------------------------------------------------------------
   0            2000.00         50.00          0.00          0.00       2000.00
   1            2000.00        900.00        400.00        500.00       1500.00
   2            1500.00        800.00        300.00        500.00       1000.00
   3            1000.00        700.00        200.00        500.00        500.00
   4             500.00        600.00        100.00        500.00          0.00

En la modelación de créditos con **`cashflow`** se consideran dos tipos de costos:

* Los puntos de descuento (dispoints) como porcentaje sobre el monto de la deuda. Estos son una forma de pago de intereses por anticipado con el fin de bajar la tasa de interés del crédito. 

* Los puntos de originación (orgpoints) como porcentaje del monto de deuda. Son los costos de constitución del crédito y no son considerados como intereses.

Ya que los intereses de los créditos pueden descontarse como costos financieros, estos disminuyen el pago del impuesto de renta. Por consiguiente, en el análisis de los créditos debe tenerse en cuenta el beneficio por pago de intereses el cual equivale a los impuestos pagados por periodo multiplicados por la tasa del impuesto de renta. Ya que los puntos de descuento son intereses, estos se tienen en cuenta en el cálculo de este beneficio.

In [5]:
## flujo de caja para el crédito antes de impuestos
credito1.toCashflow(taxRate = 0.00)

  time                Values
-----------------------------
  0                  1950.00
  1                  -900.00
  2                  -800.00
  3                  -700.00
  4                  -600.00

In [6]:
## flujo de caja para el crédito antes de impuestos
## note que no hay cambio para el valor con t=0
credito1.toCashflow(taxRate = 0.30)

  time                Values
-----------------------------
  0                  1950.00
  1                  -780.00
  2                  -710.00
  3                  -640.00
  4                  -570.00

In [7]:
## la tasa efectiva pagada por el crédito es 
## aquella que hace el valor presente cero para
## el flujo de caja anterior (antes o después de 
## impuestos)
credito1.trueRate(taxRate = 0.30)

0.15316757055069052

In [8]:
## se almacenna los datos para este credito
monto[1]   = 2000
interes[1] = credito1.trueRate(taxRate = 0.30)

In [9]:
## préstamo 2.
## -------------------------------------------------------
## 
credito2 = cf.balloonLoan(amount     = 1000,     # monto
                          rate       =    0.20,  # tasa de interés
                          life       =    4,     # número de cuotas
                          dispoints  =    0.24)  # costos de originación
credito2 


   t          Beginning         Total      Interest     Principal        Ending
              Principal       Payment       Payment       Payment     Principal
--------------------------------------------------------------------------------
   0            1000.00        240.00        240.00          0.00       1000.00
   1            1000.00        386.29        200.00        186.29        813.71
   2             813.71        386.29        162.74        223.55        590.16
   3             590.16        386.29        118.03        268.26        321.91
   4             321.91        386.29         64.38        321.91          0.00

In [10]:
credito2.toCashflow(taxRate = 0.30)

  time                Values
-----------------------------
  0                   832.00
  1                  -326.29
  2                  -337.47
  3                  -350.88
  4                  -366.97

In [11]:
credito2.trueRate(taxRate = 0.30)

0.2333658454862202

In [12]:
## se almacenna los datos para este credito
monto[2]   = 1000
interes[2] = credito2.trueRate(taxRate = 0.30)

In [13]:
## préstamo 3.
## -------------------------------------------------------
## 
credito3 = cf.bulletLoan( amount     = 5000,     # monto
                          rate       =    0.07,  # tasa de interés
                          life       =    4,     # número de cuotas
                          orgpoints  =    0.01,  # costos de originación
                          dispoints  =    0.20)  # puntos de descuento
credito3 


   t          Beginning         Total      Interest     Principal        Ending
              Principal       Payment       Payment       Payment     Principal
--------------------------------------------------------------------------------
   0            5000.00       1050.00       1000.00          0.00       5000.00
   1            5000.00        350.00        350.00          0.00       5000.00
   2            5000.00        350.00        350.00          0.00       5000.00
   3            5000.00        350.00        350.00          0.00       5000.00
   4            5000.00       5350.00        350.00       5000.00          0.00

In [14]:
credito3.toCashflow(taxRate = 0.30)

  time                Values
-----------------------------
  0                  4250.00
  1-3                -245.00
  4                 -5245.00

In [15]:
credito3.trueRate(taxRate = 0.30)

0.0959014617168048

In [16]:
## se almacenan los datos de este crédito
monto[3]   = 5000
interes[3] = credito3.trueRate(taxRate = 0.30)

In [17]:
## montos
monto

[4000, 2000, 1000, 5000]

In [18]:
## tasas
interes

[0.25, 0.15316757055069052, 0.2333658454862202, 0.0959014617168048]

In [19]:
## Costo ponderado del capital (WACC)
## -------------------------------------------------------------
## es el promdio ponderado de las tasas por 
## el porcentaje de capital correspondiente a cada fuente
##
s    = sum(monto)           # capital total
wacc = sum([x*r/s for x, r in zip(monto, interes)])
wacc

0.16826735793096878

---

[Contenido](#Contenido)